[Neural Ordinary Differential Equations \- MSur](https://msurtsukov.github.io/Neural-ODE/)

In [1]:
import numpy
numpy.linspace(0., 1., 101)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [2]:
numpy.diff(numpy.linspace(0., 1., 101))
numpy.cumsum

<function numpy.cumsum(a, axis=None, dtype=None, out=None)>

In [3]:
import torch
t = torch.linspace(0, 1, 101)
t[1:] - t[:-1]

tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100,
        0.0100])

In [4]:
import numpy
def ode_solve(z0, t0, t1, f):#t0 & t1 are not scolar?
#    print(t0, t1)
    t_array = torch.linspace(t0[0,0], t1[0,0], steps=101)
    z = z0
    for t, h in zip(t_array[1:], t_array[1:]-t_array[:-1]):
        z = z + h*f(z, t)
    return z

ode_solve(torch.tensor([0.,1.,2.]), torch.tensor([[0]]), torch.tensor([[1]]), lambda x,t : x)

tensor([0.0000, 2.7048, 5.4096])

In [5]:
import math
math.e * torch.tensor([0., 1., 2.])

tensor([0.0000, 2.7183, 5.4366])

In [6]:
import torch
from torch import nn
from torch import Tensor
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, a):
        """
        Compute f, adf/dz, adf/dp(parameter:theta) & adf/dt
        Args:
            - z:
                - shape:(batch_size, )
            - t:
            - a: adjoint
                - the length of `graph_outputs` should match the length of `outputs`
                    >[Automatic differentiation package \- torch\.autograd — PyTorch master documentation](https://pytorch.org/docs/stable/autograd.html)
        """
        batch_size = z.shape[0]
        
        #calc forward propagation
        out = self.forward(z, t)
        #calc backward propagation
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,),
            (z, t) + tuple(self.parameters()),
            grad_outputs=(a,),
            allow_unused=True,
            retain_graph=True,
        )
        #expand adfdp
        #[adfdps_of_parameter1, adfdps_of_parameter2,...] -> [*adfdp_in_batch1, *adfdp_in_batch2,...]/batchsize
        if adfdp is not None:
            adfdp = torch.cat([grad_about_a_parameter.flatten() for grad_about_a_parameter in adfdp]).unsqueeze(0)
            adfdp = adfdp.expand(batch_size, -1)#-1 should be the number of parameters
        #expand adfdt
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size

        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        return torch.cat([p.flatten() for p in self.parameters()])

In [7]:
torch.cat([torch.randn(1,3)]*3,0)

tensor([[ 0.7026,  0.8866, -0.7396],
        [ 0.7026,  0.8866, -0.7396],
        [ 0.7026,  0.8866, -0.7396]])

>We have to separate it from main `torch.nn.Module` because custom backward function can’t be implemented inside Module, but can be implemented inside `torch.autograd.Function`.


In [8]:
class ODEAdjoint(torch.autograd.Function):
    """
    Custom autograd function for neural ode
    """
    @staticmethod
    def forward(context, z0, t, flat_parameters, func):
        """
        underconstruction
        Args:
            context:
            z0:
            t (torch.Tensor): like torch.Tensor([0, 1])
            flat_parameters:return of `ODEF.flatten_parameters()`
            func (ODEF):
        Return:
            torch.Tensor: whole z 
        Examples:
            
        """
        assert isinstance(func, ODEF)#To avoid emerge type error only in backword()
        
        batch_size, *z_shape = z0.size()
        #???
        with torch.no_grad():
            z = torch.zeros(len(t), batch_size, *z_shape).to(z0)#make same type array & use same device
            z[0] = z0
            #This line does only allocate memory explicitly?
            for i in range(len(t)-1):
                z[i+1] = ode_solve(z[i], t[i], t[i+1], func)
        #save states for backward
        context.func = func
        context.save_for_backward(
            t,
            z.clone(),#why take copy?
            flat_parameters,
        )
        
        return z
    
    @staticmethod
    def backward(context, dLdz):
        """

        Args:
            dLdz (torch.Tensor):
                dLdz.shape: len(t), batch_size, *z_shape
        """
        func = context.func
        t, z, flat_parameters = context.saved_tensors
        _, batch_size, *z_shape = z.size()
        number_of_dimension = numpy.prod(z_shape)
        number_of_parameters = len(flat_parameters)

        def augmented_dynamics(aug_z_i, t_i):
            """
            Dynamics of augmented system t obe calculated backwards in time
            Args:
                arg_z_i (torch.Tensor): torch.cat(z_i, a, )
            """
            z_i, a = aug_z_i[:, :number_of_dimension], aug_z_i[:, number_of_dimension:2*number_of_dimension]  # ignore parameters and time

            # Unflatten z and a
            z_i = z_i.view(batch_size, *z_shape)
            a = a.view(batch_size, *z_shape)
            with torch.set_grad_enabled(True):
                t_i = t_i.detach().requires_grad_(True)
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, a)  # bs, *z_shape
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(batch_size, *z_shape).to(z_i)
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(batch_size, number_of_parameters).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(batch_size, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(batch_size, number_of_dimension)
            adfdz = adfdz.view(batch_size, number_of_dimension)
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)
        
        dLdz = dLdz.view(len(t), batch_size, number_of_dimension)#flatten for convinience
        with torch.no_grad():
            adjoint_z = torch.zeros(batch_size, number_of_dimension).to(dLdz)
            adjoint_p = torch.zeros(batch_size, number_of_parameters).to(dLdz)
            adjoint_t = torch.zeros(len(t), batch_size, 1).to(dLdz)
            
            for i in reversed(range(1, len(t))):# i in [batch_size, ..., 1]
                f_i = func(z[i], t[i]).view(batch_size, number_of_dimension)#flatten f_i
                dLdz_i = torch.bmm(#(a*b).sum()
                    dLdz[i].unsqueeze(-1).transpose(1, 2),#(batch_size, 1, number_of_dimension)
                    f_i.unsqueeze(-1)#(batch_size, number_of_dimension, 1)
                )[:, 0]#(batch_size, 1, 1)--[:, 0]->(batch_size, 1)

                #Adjusting adjoints with direct gradients (using Euler method?)
                adjoint_z += dLdz_i
                adjoint_t[i] -= dLdz_i
                
                #solve augmented system backward
                augmented_z = torch.cat(
                    (
                        z[i].view(batch_size, number_of_dimension),
                        adjoint_z,
                        torch.zeros(batch_size, number_of_parameters).to(z),
                        adjoint_t[i],
                    ),
                    dim=-1
                )
                augmented_answer = ode_solve(augmented_z, t[i], t[i-1], augmented_dynamics)
                adjoint_z[:] = augmented_answer[:, number_of_dimension:2*number_of_dimension]
                adjoint_p[:] = augmented_answer[:, 2*number_of_dimension:2*number_of_dimension + number_of_parameters]
                adjoint_t[i-1] = augmented_answer[: 2*number_of_dimension + number_of_parameters:]
                
                #del augmented_z, augmented_answer
            #Adjust t=0 adjoint with direct gradients (using Euler method?)
            dLdz_0 = torch.bmm(
                dLdz[0].unsqueeze(-1).transpoze(1, 2),
                func(z[0], t[0]).view(batch_size, number_of_dimension).unsqueeze(-1),#f_i.unsqueeze(-1)???
            )[:, 0]
            adjoint_z += dLdz_0
            adjoint_t[0] -= dLdz_0
        return adjoint_z.view(batch_size, *z_shape), adjoint_t, adjoint_p, None#why return additional None?

In [9]:
class NeuralODE(nn.Module):
    def __init__(self, func):
        super().__init__()
        assert isinstance(func, ODEF)
        self.func = func
    def forward(self, z0, t, return_whole_sequence=False):
        if return_whole_sequence:
            return ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)
        else:
            return ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)[-1]

In [10]:
class LinearODEF(ODEF):
    def __init__(self, W):
        super(LinearODEF, self).__init__()
        self.lin = nn.Linear(2, 2, bias=False)
        self.lin.weight = nn.Parameter(W)

    def forward(self, x, t):
        return self.lin(x)

class SpiralFunctionExample(LinearODEF):
    def __init__(self):
        super(SpiralFunctionExample, self).__init__(Tensor([[-0.1, -1.], [1., -0.1]]))

class RandomLinearODEF(LinearODEF):
    def __init__(self):
        super(RandomLinearODEF, self).__init__(torch.randn(2, 2)/2.)

class TestODEF(ODEF):
    def __init__(self, A, B, x0):
        super(TestODEF, self).__init__()
        self.A = nn.Linear(2, 2, bias=False)
        self.A.weight = nn.Parameter(A)
        self.B = nn.Linear(2, 2, bias=False)
        self.B.weight = nn.Parameter(B)
        self.x0 = nn.Parameter(x0)

    def forward(self, x, t):
        xTx0 = torch.sum(x*self.x0, dim=1)
        dxdt = torch.sigmoid(xTx0) * self.A(x - self.x0) + torch.sigmoid(-xTx0) * self.B(x + self.x0)
        return dxdt

class NNODEF(ODEF):
    def __init__(self, in_dim, hid_dim, time_invariant=False):
        super(NNODEF, self).__init__()
        self.time_invariant = time_invariant

        if time_invariant:
            self.lin1 = nn.Linear(in_dim, hid_dim)
        else:
            self.lin1 = nn.Linear(in_dim+1, hid_dim)
        self.lin2 = nn.Linear(hid_dim, hid_dim)
        self.lin3 = nn.Linear(hid_dim, in_dim)
        self.elu = nn.ELU(inplace=True)

    def forward(self, x, t):
        if not self.time_invariant:
            x = torch.cat((x, t), dim=-1)

        h = self.elu(self.lin1(x))
        h = self.elu(self.lin2(h))
        out = self.lin3(h)
        return out

In [11]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def to_np(x):
    return x.detach().cpu().numpy()
def plot_trajectories(obs=None, times=None, trajs=None, save=None, figsize=(16, 8)):
    plt.figure(figsize=figsize)
    if obs is not None:
        if times is None:
            times = [None] * len(obs)
        for o, t in zip(obs, times):
            o, t = to_np(o), to_np(t)
            for b_i in range(o.shape[1]):
                plt.scatter(o[:, b_i, 0], o[:, b_i, 1], c=t[:, b_i, 0], cmap=cm.plasma)

    if trajs is not None:
        for z in trajs:
            z = to_np(z)
            plt.plot(z[:, 0, 0], z[:, 0, 1], lw=1.5)
        if save is not None:
            plt.savefig(save)
    plt.show()

In [12]:
from torch.nn import functional as F
from torch.autograd import Variable
def conduct_experiment(ode_true, ode_trained, n_steps, name, plot_freq=10):
    # Create data
    z0 = Variable(torch.Tensor([[0.6, 0.3]]))

    t_max = 6.29*5
    n_points = 200

    index_np = numpy.arange(0, n_points, 1, dtype=numpy.int)
    index_np = numpy.hstack([index_np[:, None]])
    times_np = numpy.linspace(0, t_max, num=n_points)
    times_np = numpy.hstack([times_np[:, None]])

    times = torch.from_numpy(times_np[:, :, None]).to(z0)
    obs = ode_true(z0, times, return_whole_sequence=True).detach()
    obs = obs + torch.randn_like(obs) * 0.01

    # Get trajectory of random timespan
    min_delta_time = 1.0
    max_delta_time = 5.0
    max_points_num = 32
    def create_batch():
        t0 = numpy.random.uniform(0, t_max - max_delta_time)
        t1 = t0 + numpy.random.uniform(min_delta_time, max_delta_time)

        idx = sorted(numpy.random.permutation(index_np[(times_np > t0) & (times_np < t1)])[:max_points_num])

        obs_ = obs[idx]
        ts_ = times[idx]
        return obs_, ts_

    # Train Neural ODE
    optimizer = torch.optim.Adam(ode_trained.parameters(), lr=0.01)
    for i in range(n_steps):
        obs_, ts_ = create_batch()

        z_ = ode_trained(obs_[0], ts_, return_whole_sequence=True)
        loss = F.mse_loss(z_, obs_.detach())

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        if i % plot_freq == 0:
            z_p = ode_trained(z0, times, return_whole_sequence=True)

            plot_trajectories(obs=[obs], times=[times], trajs=[z_p], save=f"assets/imgs/{name}/{i}.png")
            clear_output(wait=True)

In [13]:
ode_true = NeuralODE(SpiralFunctionExample())
ode_trained = NeuralODE(RandomLinearODEF())

In [14]:
conduct_experiment(ode_true, ode_trained, 500, "linear")

RuntimeError: The expanded size of the tensor (1) must match the existing size (9) at non-singleton dimension 1.  Target sizes: [1, 1].  Tensor sizes: [9]